In [ ]:
from grobid.client import GrobidClient
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import xml.etree.ElementTree as ET
import os

<h1>Modelo para tokenizar</h1>

In [ ]:
model = SentenceTransformer("jamescalam/minilm-arxiv-encoder")

<h1>Obtener las clases y propiedades con grobid</h1>

In [ ]:
import json
from processor import PaperProcessor

input_path = "../res/datasets/raw/"
output_path = "../res/datasets/grobid/"

processor = PaperProcessor(output_path=output_path)
papers = processor.process_folder(input_path)

In [ ]:
papers[29].authors[0].affiliation.name

<h1>Obtener las clases y propiedades desde los xml ya procesados</h1>

In [38]:
import json
import os
from processor import PaperProcessor

input_path = "../res/datasets/raw/"
output_path = "../res/datasets/grobid/"

processor = PaperProcessor(output_path=output_path)
papers = processor.process_folder_from_xml(pdf_path=input_path)

In [39]:
papers[29].authors[0].forename
papers[29].references[0].date

'1990'

In [40]:

# Crear una lista de diccionarios con la información de cada paper
paper_list = []
for paper in papers:
    paper_dict = {}
    paper_dict['title'] = paper.title
    paper_dict['authors'] = [{'forename': author.forename,
                              'surname': author.surname,
                              'email': author.email,
                              'affiliation_name': author.affiliation.name,
                              'affiliation_country': author.affiliation.country}
                             for author in paper.authors]
    paper_dict['abstract'] = paper.abstract
    paper_dict['acknowledgements'] = str(paper.acknowledgements)
    paper_dict['references'] = [{'cites': ref.cites, 'date': ref.date} for ref in paper.references]
    paper_dict['keywords'] = paper.keywords
    paper_dict['institution'] = paper.get_institution()
    paper_dict['filename'] = paper.filename
    paper_list.append(str(paper_dict))


In [46]:
paper_list 

["{'title': 'Biological Sequence Compression Algorithms', 'authors': [{'forename': 'Toshiko', 'surname': 'Matsumoto1', 'email': 'toshikom@is.s.u-tokyo.ac.jp', 'affiliation_name': 'Department of Information Science', 'affiliation_country': 'Japan'}, {'forename': 'Kunihiko', 'surname': 'Sadakane2', 'email': None, 'affiliation_name': 'Graduate School of Information Sciences', 'affiliation_country': 'Japan'}, {'forename': 'Hiroshi', 'surname': 'Imail', 'email': 'imai@is.s.u-tokyo.ac.jp', 'affiliation_name': None, 'affiliation_country': None}], 'abstract': 'Today, more and more DNA sequences are becoming available. The information about DNA sequences are stored in molecular biology databases. The size and importance of these databases will be bigger and bigger in the future, therefore this information must be stored or communicated efficiently. Furthermore, sequence compression can be used to define similarities between biological sequences.The standard compression algorithms such as gzip o

In [48]:
   
# Crear la ruta de salida para cada archivo JSON
json_path = os.path.join("..", "res", "datasets", "json", f"{paper.filename}.json")
    
# Guardar el diccionario en un archivo JSON
with open(dir_path, "w") as f:
    json.dump(paper_dict, f)

# Escribir la lista de diccionarios en un archivo JSON
with open(os.path.join("..", "res", "datasets", "json", "papers.json"), 'w') as f:
    json.dump(paper_list, f)

PermissionError: [Errno 13] Permission denied: '../res/datasets/json/'